M Kronig (23029039)

CSC-40098 Final Project

Analysis of posts from the CS Educators Stack Exchange website
(SS3) What questions are computer science eduators asking?

In [ ]:
# Importing packages needed for project
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

(6496, 10)

In [ ]:
# Testing tthe dataframe
df = pd.read_csv('/Users/matthewkronig/Git/CSC-40098/cs_ed_data.csv')

(6496, 10)

Looking at the shape of the data, there are 6496 rows and 10 columns

In [5]:
# Looking a the shape of the data 
df.shape

(6496, 10)

Using the dataframe head() function to inspect the columns and a small set of the rows

In [6]:
df.head()

,Id,PostTypeId,Title,Body,Score,Viewcount,CommentCount,AnswerCount,Tags,CreationDate
0,2,1,What autograding software do folks use for Jav...,<p>Grading currently either takes me a huge am...,26,5395.0,9,8.0,<java><grading><auto-graders>,2017-05-23 14:20:02
1,3,1,Is it possible to ensure division of labor on ...,<p>Let's say I've assigned my Intro to CS clas...,10,598.0,1,4.0,<group-work>,2017-05-23 14:20:35
2,4,1,Objects Lab Before Inheritance and Polymorphism,<p>I am finding it very difficult to create a ...,9,711.0,3,11.0,<object-oriented-paradigm><labs><ap-computer-s...,2017-05-23 14:21:14
3,6,1,Teaching Induction to Prove the Correctness of...,<p>This is a subject that I have had a lot of ...,8,412.0,1,3.0,<lesson-ideas><cs-theory><induction>,2017-05-23 14:24:14
4,7,1,Should the first Programming/Algorithms class ...,"<p>When I was studying, my professors had diff...",22,1404.0,7,9.0,<pseudo-code>,2017-05-23 14:25:23


Using the dtypes method to have a look a the the datatypes Pandas has assigned

In [7]:
df.dtypes

Id                int64
PostTypeId        int64
Title            object
Body             object
Score             int64
Viewcount       float64
CommentCount      int64
AnswerCount     float64
Tags             object
CreationDate     object
dtype: object